In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import os
import sys

import logging
import warnings
import numpy as np
import astropy as ap
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt

import h5py
import tqdm.notebook as tqdm

import kalepy as kale
import kalepy.utils
import kalepy.plot

import holodeck as holo
import holodeck.sams
import holodeck.gravwaves
from holodeck import cosmo, utils, plot, discrete, sams, host_relations, _PATH_DATA
from holodeck.constants import MSOL, PC, YR, MPC, GYR, SPLC
from pathlib import Path

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
plt.rcParams.update({'grid.alpha': 0.5})
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode

log = holo.log
log.setLevel(logging.INFO)

# ---- Define filepath containing simulation galaxy merger data files ----#
# ---- (if using files not in _PATH_DATA) ---- #
_HOME_PATH = Path('~/').expanduser()
p = os.path.join(_HOME_PATH, 'cosmo_sim_merger_data')
if os.path.exists(p):
    _SIM_MERGER_PATH = p
else:
    p = os.path.join(_HOME_PATH, 'nanograv/cosmo_sim_merger_data')
    if os.path.exists(p):
        _SIM_MERGER_PATH = p
    else:
        _SIM_MERGER_PATH = _PATH_DATA
print(f"{_SIM_MERGER_PATH=}")
# ------------------------------------------------------------------------ #

In [ ]:
def calc_sim_merging_gsmf(dpop, mass_type='all', req_z=None, req_dz=None, mask=None, verbose=False):
    
    mstar = dpop.pop.mbulge/MSOL
    if verbose: print(f"in calc_sim_merging_gsmf: {mstar.shape=}")

    if req_z is None:
        if verbose: print("calculating gsmf for all redshifts.")
        if mask is not None:
            mstar = mstar[mask]
    else:
        if req_dz is None:
            req_dz = 0.1
        z = dpop.pop.redz
        zmask = (np.abs(z-req_z)<req_dz)
        if mask is not None:
            mstar = mstar[mask&zmask]
        else:
            mstar = mstar[zmask]
    print(f"after masking, in calc_sim_merging_gsmf: {mstar.shape=}")
        
        
    if mass_type == 'tot':
        mstar = mstar[:,0] + mstar[:,1]
    elif mass_type == 'pri':
        mstar = mstar.max(axis = 1)
    elif mass_type not in ('tot','pri','all'):
        err = "`mass_type` must be 'tot', 'pri', or 'all'"
        raise ValueError(err)

    if verbose: print(f"after setting mass_type, in calc_sim_merging_gsmf: {mstar.shape=}")

    mstar = np.log10(mstar)
    box_vol_mpc = dpop.pop._sample_volume / (1.0e6*PC)**3

    mhist, mbin_edges = np.histogram(mstar, range=(7.25,15.25), bins=16)
    mbinsize = mbin_edges[1]-mbin_edges[0]
    mbins = mbin_edges[:-1]+mbinsize
        
    #return mbins, mhist/mbinsize/np.log(10)/box_vol_mpc, mhist
    return mbins, mhist/mbinsize/box_vol_mpc, mhist

def compare_sim_merging_gsmfs(dpops, req_z=None, req_dz=None):
    
    assert isinstance(dpops, list), '`dpops` must be a list of binary populations'

    fig, ax = plt.subplots(figsize=[10, 5])
    ax.set(ylabel='Probability Density')
    ax.grid(alpha=0.01)
    ax.set(yscale='log')
    
    if req_z is None:
        lbl_extra = ' (mergers, all z)'
    else:
        lbl_extra = f' (mergers, {np.maximum(0,req_z-req_dz):.2g}<z<{req_z+req_dz:.2g})'
    
    for i,dp in enumerate(dpops):
        xlab = r'$\log_{10}(M_{*})$' 
        ax.set(xlabel=xlab)
        ax.set(ylabel=r'GSMF [$(dex^{-1} Mpc^{-3}$]')

        mbins, gsmf, mhist = calc_sim_merging_gsmf(dp, req_z=req_z, req_dz=req_dz)
        plt.plot(mbins, gsmf, ':', alpha=0.5, color=dp.color, lw=dp.lw)
        plt.plot(mbins[mhist>10], gsmf[mhist>10], color=dp.color, lw=dp.lw,label=dp.lbl+lbl_extra)

    return fig, ax


In [ ]:
import compare_discrete

In [ ]:
tmp = compare_discrete.create_dpops(allow_mbh0=True, mod_mmbulge=False, skip_evo=True, fpath=_SIM_MERGER_PATH)
all_dpops, tng_dpops, all_fsa_dpops, tng_fsa_dpops = tmp

In [ ]:
print("creating SAM using Galaxy Pair Fraction (GPF) + Galaxy Merger Timescale (GMT)...")

sam = sams.Semi_Analytic_Model(gpf = sams.GPF_Power_Law())

print("SKIPPING HARDENING")

#print("    ...calculating hardening")
#hard = holo.hardening.Fixed_Time_2PL_SAM(sam, all_fsa_dpops[0].tau, sepa_init=1.0e4*PC)
#print("    ...creating gwb")
#gwb_sam = sam.gwb_new(all_fsa_dpops[0].freqs_edges, hard, realize=500)

In [ ]:
print("creating 'no-GPF' SAM using Galaxy Merger Rate (GMR) "
      "    (uses galaxy merger rates directly from RG15 instead of GPF+GMT)...")

sam_no_gpf = sams.Semi_Analytic_Model()

print("SKIPPING HARDENING")

#print("    ...calculating hardening for no-GPF SAM")
#hard_no_gpf = holo.hardening.Fixed_Time_2PL_SAM(sam_no_gpf, all_fsa_dpops[0].tau, sepa_init=1.0e4*PC)
#print("    ...creating gwb for no-GPF SAM")
#gwb_sam_no_gpf = sam_no_gpf.gwb_new(all_fsa_dpops[0].freqs_edges, hard_no_gpf, realize=50)

In [ ]:
compare_sim_merging_gsmfs(all_dpops)
plt.legend()
plt.show()

In [ ]:
compare_sim_merging_gsmfs(all_fsa_dpops)
plt.legend()
plt.show()

In [ ]:
compare_sim_merging_gsmfs(tng_dpops)
plt.legend()
plt.show()

In [ ]:
nrows = 2
ncols = 2
mass_type = 'tot'
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, sharex='all', sharey='all', figsize=[10,9])
#print(axes.shape)
#print(axes.dtype)
#print(axes[0].shape)

if mass_type == 'pri':
    xlab = r'$\log_{10}(M_{*,pri})$'
elif mass_type == 'tot':
    xlab = r'$\log_{10}(M_{*,tot})$'
elif mass_type == 'all':
    xlab = r'$\log_{10}(M_{*}) (all progenitors)$'

axes[0,0].set(yscale='log', 
              ylabel=r'GSMF [$(\log_{10} M)^{-1} Mpc^{-3}$]',
              xlim=(7.25,15.5), ylim=(1.0e-6,1.0))
axes[1,0].set(yscale='log',xlabel=xlab,
              ylabel=r'GSMF [$(\log_{10} M)^{-1} Mpc^{-3}$]',
              xlim=(7.25,15.5), ylim=(1.0e-6,1.0))
axes[1,1].set(yscale='log',xlabel=xlab,
              #ylabel=r'BHMF [$(\log_{10} M)^{-1} Mpc^{-3}$]',
              xlim=(7.25,15.5), ylim=(1.0e-6,1.0))
    
#dpop_lists = [t50_hires_fid_and_fsa_dpops, t100_hires_fid_and_fsa_dpops, ill_hires_fid_and_fsa_dpops]
#dpop_lists = [ 
#    [ d for d in all_hires_fid_and_fsa_dpops if d.lbl in ['TNG50-1', 'fsa-TNG50-1', 'fsa-TNG50-1-bh0'] ],
#    [ d for d in all_hires_fid_and_fsa_dpops if d.lbl in ['TNG50-1-N100', 'fsa-TNG50-1-N100', 'fsa-TNG50-1-N100-bh0'] ],
#    [ d for d in all_hires_fid_and_fsa_dpops if d.lbl in ['TNG100-1', 'fsa-TNG100-1', 'fsa-TNG100-1-bh0'] ],
#    ill_hires_fid_and_fsa_dpops
#    ]
dpop_lists = [d for d in [all_fsa_dpops]]
print(len(dpop_lists))
for i in range(nrows):
    for j in range(ncols):
        
        if i+2*j >= np.minimum((nrows*ncols),len(dpop_lists)): 
            break

        for dp in dpop_lists[i+2*j]:
            qbulge = dp.pop.mbulge[:,0] / dp.pop.mbulge[:,1]
            qbulge[qbulge>1] = 1.0/qbulge[qbulge>1]
            #print(f"{qbulge.min()=} {qbulge.max()=}")
            
            x,y,mh = calc_sim_merging_gsmf(dp, mass_type=mass_type)
            axes[i,j].plot(x,y, lw=dp.lw, label=dp.lbl)
            
            xqcut,yqcut,mhqcut = calc_sim_merging_gsmf(dp, mask = (qbulge>0.1), 
                                                       mass_type=mass_type)
            axes[i,j].plot(xqcut,yqcut, lw=dp.lw, label=dp.lbl+' q>0.1', alpha=0.4)
            
            axes[i,j].legend()

    else:
        continue  # only executed if the inner loop did NOT break
    break  # only executed if the inner loop DID break

fig.suptitle(f"GSMF for merging galaxies (mass type = {mass_type})")
fig.subplots_adjust(top=0.95, wspace=0.1, hspace=0.1)
plt.show()

### Load data for all subhalos & compare with merging subhalos

In [ ]:
def load_sim_gsmf_file(dpop, basePath):

    for s in ['Ill-1', 'TNG50-1', 'TNG100-1', 'TNG300-1']:
        if s in dpop.lbl: 
            print(f"{s=}")
            gsmf_fpath = os.path.join(basePath, f"gsmf_all_snaps_Nmin1_{s}.hdf5")
            print(f"{gsmf_fpath=}")
    try:                
        f = h5py.File(gsmf_fpath,"r")
    except:
        raise Exception(f"Could not open GSMF file {gsmf_fpath}.")

    return f

def calc_sim_gsmf_from_snaps(dpop, req_z, req_binsize=0.05, verbose=False):
    

    f = load_sim_gsmf_file(dpop, dpop.basepath)

    box_vol_mpc = f.attrs['box_volume_mpc']
    snapnums = f.attrs['SnapshotNums']
    scalefacs = f.attrs['SnapshotScaleFacs']
    zsnaps = 1.0 / scalefacs - 1.0
    
    diff = np.abs(zsnaps-req_z)
    snapNum = snapnums[diff==diff.min()][0]
    zsnap = zsnaps[diff==diff.min()][0]
    if verbose or (diff.min()>0.01):
        print(f"{req_z=}, {snapNum=}, {zsnap=}, {diff.min()=}")

    dlgm_orig = f.attrs['LogMassBinWidth']
    mbin_edges_orig = np.array(f['StellarMassBinEdges'])
    nbins_orig = mbin_edges_orig.size - 1
    mhist_all_snaps = np.array(f['StellarMassHistograms'])
    
    mhist_snap_orig = mhist_all_snaps[:,(snapnums==snapNum)].flatten()
    if verbose: print(f"{mhist_snap_orig.shape=}, {mbin_edges_orig.shape=}")
    if mhist_snap_orig.size != nbins_orig:
        print('whoops')
        return


    if req_binsize < dlgm_orig:
        raise ValueError(f"{req_binsize=} requested, but min allowed is {dlgm_orig=}")
    if int(req_binsize/dlgm_orig) > nbins_orig/2:
        raise ValueError(f"{req_binsize=} requested, but max allowed is {dlgm_orig*nbins_orig/2=}")

    ncomb = int(req_binsize/dlgm_orig)
    dlgm = dlgm_orig * ncomb
    mbin_edges = mbin_edges_orig[::ncomb]
    nbins = mbin_edges.size
    if ncomb > 1:
        mbin_edges = np.append(mbin_edges, mbin_edges[-1]+dlgm)
        mhist_snap = np.zeros((nbins))
        if verbose: print(f"{mbin_edges.size=}")
        for i in range(mbin_edges.size-1):
            mhist_snap[i] = mhist_snap_orig[i*ncomb:i*ncomb+ncomb].sum()
        if verbose:
            print(f"{mbin_edges_orig=}")
            print(f"{mbin_edges=}")
    else:
        if verbose:
            print(f"WARNING: {req_binsize=}, {ncomb=}; retaining original binsize {dlgm_orig=}")
        assert mbin_edges.all() == mbin_edges_orig.all() and dlgm == dlgm_orig, "Error in setting ncomb=1!"
        mhist_snap = mhist_snap_orig
        
    if verbose:
        print(f"{mhist_all_snaps.shape=}, {mhist_all_snaps.min()=}, {mhist_all_snaps.max()=}")
        print(f"{mhist_snap.shape=}, {mhist_snap.min()=}, {mhist_snap.max()=}")
        print(f"{snapnums=}")
        print(f"{dlgm=}, {mbin_edges.shape=}")
        print(f"{mbin_edges=}")

    ##gsmf = mhist_snap / dlgm / np.log(10) / box_vol_mpc  # dex^-1 Mpc^-3
    gsmf = mhist_snap / dlgm / box_vol_mpc  # dex^-1 Mpc^-3
    return mbin_edges[:-1]+0.5*dlgm, gsmf, mhist_snap #mbin_edges, dlgm


In [ ]:
print([d.lbl for d in all_fsa_dpops])

In [ ]:
##tng100_basePath = '/orange/lblecha/IllustrisTNG/Runs/TNG100-1/output/'
##tng50_basePath = '/orange/lblecha/IllustrisTNG/Runs/TNG50-1/output/'
##ill_basePath = '/orange/lblecha/Illustris/Illustris-1/output/'
#basePath = '/Users/lblecha/holodeck/holodeck/data/'
#tng100_basePath = basePath
#tng50_basePath = basePath
#ill_basePath = basePath

z_arr = np.array([0, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.5, 0.75, 1, 1.5, 2, 3, 4, 5])
binsize = 0.2
gsmf_sam = {}

sam_lgmstar_arr = np.arange(7.0,13.1,0.05)
sam_mstar_arr = 10**sam_lgmstar_arr * MSOL

for z in z_arr:
    for d in all_dpops:
        print(f"calculating GSMF at {z=} for {d.lbl=}")
        d.calc_sim_gsmf_from_snaps(z, req_binsize=binsize)
        print(f"{d.mhist_bins=}")
        print(f"{d.gsmf=}")
        print(f"{d.mhist=}")

    gsmf_sam[z] = sam._gsmf(sam_mstar_arr, z) ##/ np.log(10)  # units of dex^-1 Mpc^-3
print(f"{gsmf_sam[1]=}")
print(f"{sam_lgmstar_arr=}")

In [ ]:
plt.yscale('log')
#plt.plot(bins_tng100[0.5], gsmf_tng100[0.5],'c--',label='TNG100-1 (all gals, z=0.5)')
#plt.plot(bins_tng100[5], gsmf_tng100[5],'c--',label='TNG100-1 (all gals, z=5)')
for z in [0.5, 5]:
    plt.plot(all_dpops[0].mhist_bins[z], all_dpops[0].gsmf[z],'c--',label=f'{all_dpops[0].lbl} (all gals, z={z})')


In [ ]:
mstar_pri,mstar_rat,mstar_tot,redz = sam.mass_stellar() 
print(mstar_pri.shape)
print(mstar_pri.min()/MSOL, mstar_pri.max()/MSOL)

print(f"phi0={sam._gsmf._phi0}, phiz={sam._gsmf._phiz}")
print(f"log10(mchar0/msun)={np.log10(sam._gsmf._mchar0/MSOL):g}, log10(mcharz/msun)={np.log10(sam._gsmf._mcharz/MSOL):g}")
print(f"alpha0={sam._gsmf._alpha0}, alphaz={sam._gsmf._alphaz}")
#plt.xscale('log')
plt.yscale('log')
#plt.xlim(5.0e7,2.0e12)
plt.ylim(1.0e-6,0.01)
plt.plot(sam_lgmstar_arr, gsmf_sam[0], lw=3)
plt.plot(sam_lgmstar_arr, gsmf_sam[0.5], lw=3)
plt.plot(sam_lgmstar_arr, gsmf_sam[5], lw=3)
#plt.plot(mstar_pri.flatten()/MSOL, sam._gsmf(mstar_pri.flatten(), 0.0)/ np.log(10) ,'.',lw=0,ms=0.1,alpha=0.2)
#plt.plot(mstar_pri.flatten()/MSOL, sam._gsmf(mstar_pri.flatten(), 5.0)/ np.log(10) ,'.',lw=0,ms=0.1,alpha=0.2)
#phi_check_z0 = np.power(10.0, sam._gsmf._phi0 + sam._gsmf._phiz * 0.0)
#m0_check = sam._gsmf._mchar0 / MSOL
#plt.plot([1.0e8,1.0e12], [phi_check_z0, phi_check_z0])
#plt.plot([m0_check,m0_check],[1.0e-6,1.0e-2],)
print(f"{gsmf_sam[1]=}")
print(f"{sam_lgmstar_arr=}")

In [ ]:
ab = {'Perez-Gonzalez+08': 'PG08', 'Muzzin+13': 'M13'}
print(ab)
print(ab['Perez-Gonzalez+08'])
print(ab['Muzzin+13'])

In [ ]:
import parse_conselice2016_table1
c16_zlo, c16_zhi, c16_lgm, c16_gsmfs, c16_lim, refs, short_refs = parse_conselice2016_table1.calc_c16_gsmfs()
c16_zcen = (c16_zhi - c16_zlo) / 2 + c16_zlo
for i in range(len(c16_zcen)):
    print(f"zcen={c16_zcen[i]} ({c16_zlo[i]}-{c16_zhi[i]}) {refs[i]}")
print(c16_zlo.shape, c16_zhi.shape, c16_zcen.shape, c16_lgm.shape, c16_gsmfs.shape, c16_lim.shape, refs.shape)

In [ ]:
print([d.lbl for d in all_dpops])
print([d.lbl for d in all_dpops[1::2]])

In [ ]:
def compare_sim_vs_sam_gsmfs(dpops, z=0, dz=0.1, c16_zrange=None, c16_zrange_strict=False,
                             apply_mlim=False, plot_sam=True, plot_mergers=True, save=False, 
                             verbose=False): 

    #ax.set(ylabel='GSMF')
    #ax.grid(alpha=0.01)
    #ax.set(yscale='log')

    plt.rcParams.update({'font.size': 14})
    fig, ax = plt.subplots(figsize=[8.5, 6])
    ax.set(ylabel=r'GSMF [$(dex^{-1} Mpc^{-3}$]')
    xlab = r'$\log_{10}(M_{*})$' 
    ax.set(xlabel=xlab)    
    ax.grid(alpha=0.01)
    ax.set(yscale='log')
    plt.rcParams.update({'font.size': 9})

    if plot_sam:
        plt.plot(sam_lgmstar_arr, gsmf_sam[z],'k',lw=3,label=f'SAM (z={z:.2g})',alpha=0.8)
    
    lgm_min = None
    for d in dpops:
        bins = d.mhist_bins[z]
        gsmf = d.gsmf[z]
        mhist = d.mhist[z]
        mk = '--' if plot_sam and c16_zrange is not None else '-'
        plt.plot(bins, gsmf, mk, color=d.color, alpha=0.2,lw=d.lw-0.25)
        if apply_mlim:
            d.sim_mass_resolution() # get baryonic mass resolution for this sim (stored in d.mres_baryon)
            lgm_min = np.log10(d.mres_baryon)+2
            plt.plot(bins[(mhist>10)&(bins>lgm_min)], gsmf[(mhist>10)&(bins>lgm_min)], 
                     mk, color=d.color, label=f'{d.lbl} (all, z={z:.2g})',lw=d.lw-0.25)
        else:
            plt.plot(bins[mhist>=10], gsmf[mhist>=10], mk, color=d.color,
                     label=f'{d.lbl} (all, z={z:.2g})',lw=d.lw-0.25)


    if plot_mergers:
        if z is None:
            lbl_extra = ' (mergers, all z)'
        else:
            lbl_extra = f' (mrg, z={np.maximum(0,z-dz):.2g}-{z+dz:.2g})'

        for i,dp in enumerate(dpops):
            bins_mrg, gsmf_mrg, mhist_mrg = calc_sim_merging_gsmf(dp, req_z=z, req_dz=dz)
            plt.plot(bins_mrg, gsmf_mrg, ':', alpha=0.2, color=dp.color, lw=dp.lw-0.5)
            if apply_mlim:
                if lgm_min is None:
                    d.sim_mass_resolution() # get baryonic mass resolution for this sim (stored in d.mres_baryon)
                    lgm_min = np.log10(d.mres_baryon)+2
                plt.plot(bins_mrg[(mhist_mrg>10)&(bins_mrg>lgm_min)], gsmf_mrg[(mhist_mrg>10)&(bins_mrg>lgm_min)], 
                         ':', color=dp.color, lw=dp.lw-0.5,label=dp.lbl+lbl_extra)
            else:
                plt.plot(bins_mrg[mhist_mrg>10], gsmf_mrg[mhist_mrg>10], ':', color=dp.color, 
                         lw=dp.lw-0.5,label=dp.lbl+lbl_extra)
            

    plt.yscale('log')
    plt.xlim(7.0,13.2)
    plt.ylim(1.0e-8,0.1)
    
    if isinstance(c16_zrange, (tuple, list, np.ndarray)) and len(c16_zrange)==2:
        c16_lines = []
        
        c16_zlo, c16_zhi, c16_lgm, c16_gsmfs, c16_lim, refs, short_refs = parse_conselice2016_table1.calc_c16_gsmfs()
        c16_zcen = (c16_zhi - c16_zlo) / 2 + c16_zlo
        if c16_zrange_strict:
            ix = np.where((c16_zlo>=c16_zrange[0])&(c16_zhi<=c16_zrange[1]))[0]
        else:
            ix = np.where((c16_zcen>=c16_zrange[0])&(c16_zcen<=c16_zrange[1]))[0]
        if apply_mlim:
            # plot gray dashed line to extrapolate below lower mass limit of empirical GSMFs
            plt.plot(c16_lgm, c16_gsmfs[:, ix],ls='--',lw=2,alpha=0.2,color='gray')
            for ii in ix:
                plt.plot(c16_lgm[c16_lgm>=c16_lim[ii]], c16_gsmfs[c16_lgm>=c16_lim[ii], ii],
                         label=f"{short_refs[ii]} (z={c16_zlo[ii]}-{c16_zhi[ii]})")
        else:
            # don't show the lower mass limit of empirical GSMFs
            for ii in ix:
                plt.plot(c16_lgm, c16_gsmfs[:, ii],ls='--',alpha=0.8,
                         label=f"{short_refs[ii]} (z={c16_zlo[ii]}-{c16_zhi[ii]})")

    elif c16_zrange is not None:
        raise ValueError(f"Error: c16_zrange must be a len-2 tuple, list, or array. {c16_zrange=}")


    plt.legend(loc='lower left')

    if save:
        plt.savefig(f'gsmf_sim_vs_sam_z{z:.2g}.png', dpi=300)

    plt.show()

# fig, ax = plt.subplots()
#line1, = ax.plot([1, 2, 3], label="Line 1", linestyle='--')
#line2, = ax.plot([3, 2, 1], label="Line 2", linewidth=4)

## Create a legend for the first line.
#first_legend = ax.legend(handles=[line1], loc='upper right')

## Add the legend manually to the Axes.
#ax.add_artist(first_legend)

## Create another legend for the second line.
#ax.legend(handles=[line2], loc='lower right')

#plt.show()
        


In [ ]:
compare_sim_vs_sam_gsmfs(all_dpops[1::2], z=0.25, dz=0.1,c16_zrange=(.15,0.35),apply_mlim=True, save=True) #, plot_mergers=False)
#plt.plot(c16_lgm, c16_gsmfs[:,c16_zcen<0.5])

In [ ]:
compare_sim_vs_sam_gsmfs(all_dpops, z=0.25, dz=0.1, plot_sam=False,apply_mlim=True) #, plot_mergers=False)

In [ ]:
compare_sim_vs_sam_gsmfs(all_dpops[1::2], z=0.35, dz=0.1, c16_zrange=(0.3,0.4), apply_mlim=True, save=True)

In [ ]:
compare_sim_vs_sam_gsmfs(all_dpops, z=0.35, dz=0.1, plot_sam=False, apply_mlim=True) #, save=True)

In [ ]:
compare_sim_vs_sam_gsmfs(all_dpops[1::2], z=0.5, dz=0.05, c16_zrange=(0.45,0.55),apply_mlim=True, save=True) #,plot_mergers=False)

In [ ]:
compare_sim_vs_sam_gsmfs(all_dpops, z=0.5, dz=0.05, plot_sam=False,apply_mlim=True) #,plot_mergers=False)

In [ ]:
compare_sim_vs_sam_gsmfs(all_dpops[1::2], z=0.75, dz=0.1,c16_zrange=(0.65,0.85),apply_mlim=True, save=True)  #, plot_mergers=False)

In [ ]:
compare_sim_vs_sam_gsmfs(all_dpops, z=0.75, dz=0.1,plot_sam=False,apply_mlim=True)  #, plot_mergers=False)

In [ ]:
compare_sim_vs_sam_gsmfs(all_dpops[1::2], z=1, dz=0.1, c16_zrange=(0.9,1.1), apply_mlim=True, save=True) # plot_mergers=False)

In [ ]:
compare_sim_vs_sam_gsmfs(all_dpops, z=1, dz=0.1, plot_sam=False, apply_mlim=True, save=False) # plot_mergers=False)

In [ ]:
compare_sim_vs_sam_gsmfs(all_dpops[1::2], z=1.5, dz=0.1, c16_zrange=(1.4,1.6), apply_mlim=True, save=True) # plot_mergers=False)

In [ ]:
compare_sim_vs_sam_gsmfs(all_dpops, z=1.5, dz=0.1, plot_sam=False, apply_mlim=True, save=False) # plot_mergers=False)

In [ ]:
compare_sim_vs_sam_gsmfs(all_dpops[1::2], z=2, dz=0.25, c16_zrange=(1.8,2.2), apply_mlim=True, save=True) #plot_mergers=False, 

In [ ]:
compare_sim_vs_sam_gsmfs(all_dpops, z=2, dz=0.25, plot_sam=False, apply_mlim=True, save=False) #plot_mergers=False, 

In [ ]:
compare_sim_vs_sam_gsmfs(all_dpops[1::2], z=3, dz=0.25, c16_zrange=(2.75,3.25),apply_mlim=True, save=True) #plot_mergers=False,

In [ ]:
compare_sim_vs_sam_gsmfs(all_dpops, z=3, dz=0.25, plot_sam=False,apply_mlim=True, save=False) #plot_mergers=False,

In [ ]:
compare_sim_vs_sam_gsmfs(all_dpops[1::2], z=4, dz=0.25, c16_zrange=(3.75,4.25),apply_mlim=True,save=True)

In [ ]:
compare_sim_vs_sam_gsmfs(all_dpops, z=4, dz=0.25, plot_sam=False,apply_mlim=True,save=False)